In [1]:
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.optim import AdamW
from xgboost import XGBClassifier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pickle as pkl
import scipy
import os

from torch.nn import Linear, ReLU, Dropout
from torch.nn.functional import relu
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

import gurobipy as gb

In [2]:
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = (
        False  # Force cuDNN to use a consistent convolution algorithm
    )
    torch.backends.cudnn.deterministic = (
        True  # Force cuDNN to use deterministic algorithms if available
    )
    torch.use_deterministic_algorithms(
        True
    )  # Force torch to use deterministic algorithms if available


In [3]:
try:
    corlat_dataset = pkl.load(open("Data/corlat/corlat_preprocessed.pickle", "rb"))
except:
    # move dir to /ibm/gpfs/home/yjin0055/Project/DayAheadForecast
    os.chdir("/ibm/gpfs/home/yjin0055/Project/DayAheadForecast")
    corlat_dataset = pkl.load(open("Data/corlat/corlat_preprocessed.pickle", "rb"))

In [4]:
# for each solution convert the dictionary to a list of values
solutions = [
    list(corlat_dataset[i]["solution"].values())
    for i in range(len(corlat_dataset))
]

In [5]:
# convert solutions_list to numpy array
solutions = np.array(solutions)

In [6]:
model_files = os.listdir("instances/mip/data/COR-LAT")

In [7]:
# get the indices of the binary variables
indices = []
for i in range(len(corlat_dataset)):
    indices.append(list(corlat_dataset[i]["solution"].keys()))

In [8]:
# convert indices to numpy array
indices = np.array(indices)

In [9]:
# read X_train, X_test, y_train, y_test from Data/corlat/ using numpy.load
X_train = np.load("Data/corlat/X_train.npy")
X_test = np.load("Data/corlat/X_test.npy")
y_train = np.load("Data/corlat/y_train.npy")
y_test = np.load("Data/corlat/y_test.npy")

In [10]:
# train and test indices
train_indices = np.load("Data/corlat/train_idx.npy")
test_indices = np.load("Data/corlat/test_idx.npy")

In [11]:
# load the xgboost model
with open("Models/Tabular/xgboost_model_corlat.pkl", "rb") as f:
    xgb_model = pkl.load(f)

In [12]:
y_pred = xgb_model.predict(X_test)

In [13]:
y_test = y_test.astype(np.int)
print("F1 score: ", f1_score(y_test, y_pred, average="micro"))
print("Precision: ", precision_score(y_test, y_pred, average="micro"))
print("Recall: ", recall_score(y_test, y_pred, average="micro"))

F1 score:  0.9237003241685676
Precision:  0.9232569302772111
Recall:  0.9241441441441441


/tmp/ipykernel_22975/3270043010.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_test = y_test.astype(np.int)


In [14]:
# save xgboost model
xgb_model.save_model("Models/Tabular/xgboost_model_corlat.json")

# now test feasibility of the solutions

In [15]:
# basic optimization solving time
firstInstanceTest = gb.read("instances/mip/data/COR-LAT/" + model_files[test_indices[0]])
firstInstanceTest.Params.Threads = 1
firstInstanceTest.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Read LP format model from file instances/mip/data/COR-LAT/cor-lat-2f+r-u-10-10-10-5-100-3.462.b208.000000.prune2.lp
Reading time = 0.01 seconds
obj: 470 rows, 466 columns, 1751 nonzeros
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen Threadripper 1920X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 1 threads

Optimize a model with 470 rows, 466 columns and 1751 nonzeros
Model fingerprint: 0x3db7ce21
Variable types: 366 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 17 rows and 9 columns
Presolve time: 0.00s
Presolved: 453 rows, 457 columns, 1715 nonzeros
Variable types: 360 continuous, 97 integer 

In [16]:
# we are going to use first instance of test data
firstInstanceTest = gb.read("instances/mip/data/COR-LAT/" + model_files[test_indices[0]])
firstInstanceTest.Params.Threads = 1

# get indices of binary variables
firstInstanceTestBinaryIndices = indices[test_indices[0]]

# for this first instance of test data, we are going to use the xgboost prediction and fix the binary variables' values
# to the values predicted by xgboost

# get predictions from xgboost model
xgb_pred = xgb_model.predict(X_test[0].reshape(1, -1)).reshape(-1)

# get variables from the model
modelVars = firstInstanceTest.getVars()

# need to relax the binary variables to continuous variables with bounds of 0 and 1, we can use the setAttr method to change their vtype attribute
for i in range(len(firstInstanceTestBinaryIndices)):
    modelVars[firstInstanceTestBinaryIndices[i]].setAttr("VType", "C")

    # for each index in firstInstanceTestBinaryIndices, set the value of the corresponding variable to the value predicted by xgboost
    modelVars[firstInstanceTestBinaryIndices[i]].setAttr("LB", xgb_pred[i])
    modelVars[firstInstanceTestBinaryIndices[i]].setAttr("UB", xgb_pred[i])
    
# After relaxing or fixing the binary variables, we can compute the IIS as before
firstInstanceTest.computeIIS()
# # Print the conflicting variables and constraints
# for v in firstInstanceTest.getVars():
#   if v.IISLB > 0 or v.IISUB > 0:
#     print(v.varName, "is part of the IIS")
# for c in firstInstanceTest.getConstrs():
#   if c.IISConstr > 0:
#     print(c.ConstrName, "is part of the IIS")

# only assign the predicted variables that are not in the IIS to warm start the model
for i, v in enumerate(firstInstanceTest.getVars()):
    if v.IISLB == 0 and v.IISUB == 0:
        if i in firstInstanceTestBinaryIndices:
            # print(v.varName, "is not part of the IIS")
            v.setAttr("VType", "B")
            v.setAttr("LB", 0)
            v.setAttr("UB", 1)
            v.setAttr("Start", xgb_pred[i])
            
    
    # else if the variable is in the IIS, 
    # get the relaxed variable and 
    # set the bounds to 0 and 1 for the relaxed binary variables
    else:
        if i in firstInstanceTestBinaryIndices:
            # print(v.varName, "is part of the IIS")
            v.setAttr("VType", "B")
            v.setAttr("LB", 0)
            v.setAttr("UB", 1)
            

firstInstanceTest.optimize()

Read LP format model from file instances/mip/data/COR-LAT/cor-lat-2f+r-u-10-10-10-5-100-3.462.b208.000000.prune2.lp
Reading time = 0.01 seconds
obj: 470 rows, 466 columns, 1751 nonzeros
Set parameter Threads to value 1
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.9700000e+02   2.880000e+02   0.000000e+00      0s

IIS computed: 56 constraints and 73 bounds
IIS runtime: 0.01 seconds (0.00 work units)
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD Ryzen Threadripper 1920X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 1 threads

Optimize a model with 470 rows, 466 columns and 1751 nonzeros
Model fingerprint: 0x499154c7
Variable types: 366 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]

User MIP start did not pr

In [17]:
# # we are going to use first instance of test data
# firstInstanceTest = gb.read("instances/mip/data/COR-LAT/" + model_files[test_indices[0]])
# firstInstanceTest.Params.Threads = 1

# # get indices of binary variables
# firstInstanceTestBinaryIndices = indices[test_indices[0]]

# # for this first instance of test data, we are going to use the xgboost prediction and fix the binary variables' values
# # to the values predicted by xgboost

# # get predictions from xgboost model
# xgb_pred = xgb_model.predict(X_test[0].reshape(1, -1)).reshape(-1)

# # get variables from the model
# modelVars = firstInstanceTest.getVars()

# # need to relax the binary variables to continuous variables with bounds of 0 and 1, we can use the setAttr method to change their vtype attribute
# for i in range(len(firstInstanceTestBinaryIndices)):
#     modelVars[firstInstanceTestBinaryIndices[i]].setAttr("VType", "C")

#     # for each index in firstInstanceTestBinaryIndices, set the value of the corresponding variable to the value predicted by xgboost
#     modelVars[firstInstanceTestBinaryIndices[i]].setAttr("LB", xgb_pred[i])
#     modelVars[firstInstanceTestBinaryIndices[i]].setAttr("UB", xgb_pred[i])
    
# # After relaxing or fixing the binary variables, we can compute the IIS as before
# firstInstanceTest.computeIIS()

# # for each constraint, if constraint in IIS, get the slack value. Implement this.
# for c in firstInstanceTest.getConstrs():
#     if c.IISConstr > 0:
#         print(c.ConstrName, "is part of the IIS")
#         print(c.Slack)

In [18]:
# firstInstanceTest.getVars()[0]

In [19]:
# # only assign the predicted variables that are not in the IIS to warm start the model
# for i, v in enumerate(firstInstanceTest.getVars()):
#     if v.IISLB == 0 and v.IISUB == 0:
#         if i in firstInstanceTestBinaryIndices:
#             print(v.varName, "is not part of the IIS")
#             v.setAttr("VType", "B")
#             v.setAttr("LB", 0)
#             v.setAttr("UB", 1)
            
    
#     # else if the variable is in the IIS, 
#     # get the relaxed variable and 
#     # set the bounds to 0 and 1 for the relaxed binary variables
#     else:
#         if i in firstInstanceTestBinaryIndices:
#             print(v.varName, "is part of the IIS")
#             v.setAttr("VType", "B")
#             v.setAttr("LB", 0)
#             v.setAttr("UB", 1)

In [20]:
# # continue solving the model
# firstInstanceTest.optimize()

In [31]:
# the weights for each variable in the loss function should take the form of
# w_{ij} = exp(-c_i^T x^{i, j}) / sum(exp(-c_i^T x^{i, k})) for k = 1, ..., N_i
# where c_i is the vector of cost coefficient for training instance i, j is the index of the training instance, and N_i is the number of training instances


# we are going to use first instance of test data
# firstInstanceTest = gb.read("instances/mip/data/COR-LAT/" + model_files[test_indices[0]])

def custom_obj(model_files: list, indices, train_indices, y_true: np.ndarray, y_pred: np.ndarray):
    
    instance_weights = []
    
    gurobi_env = gb.Env()
    gurobi_env.setParam("OutputFlag", 0)
    
    # convert logit predictions to probabilities
    y_pred = 1.0 / (1.0 + np.exp(-y_pred))

    # convert predictions of N_samples, N_variables to binary
    y_pred_binary = np.where(y_pred > 0.5, 1, 0)
    
    # Compute the weights for each training instance
    for i in range(y_true.shape[0]):
        
        model = gb.read("instances/mip/data/COR-LAT/" + model_files[train_indices[i]], env=gurobi_env)
        
        modelVars = model.getVars()
        
        instanceBinaryIndices = indices[train_indices[i]]

        # need to relax the binary variables to continuous variables with bounds of 0 and 1, we can use the setAttr method to change their vtype attribute
        for j in range(len(instanceBinaryIndices)):
            modelVars[instanceBinaryIndices[j]].setAttr("VType", "C")

            # for each index in firstInstanceTestBinaryIndices, set the value of the corresponding variable to the value predicted by xgboost
            modelVars[instanceBinaryIndices[j]].setAttr("LB", y_pred_binary[i, j])
            modelVars[instanceBinaryIndices[j]].setAttr("UB", y_pred_binary[i, j])
        
        
        # Compute the IIS to find the list of violated constraints and variables
        model.computeIIS()
        
        # Initialize the weights
        weights = np.zeros_like(y_true[i])
        
        c = model.getAttr("Obj", model.getVars())
        
        # get violated variables indices
        violated_vars_indices = [k for k, v in enumerate(model.getVars()) if (v.IISLB > 0 or v.IISUB > 0) and k in instanceBinaryIndices]
        
        for j, v in enumerate(model.getVars()):
            # not violated
            if (v.IISLB == 0 and v.IISUB == 0) and j in instanceBinaryIndices:
                weights[j] = np.exp(np.dot(c[j], y_pred_binary[i, j]))
        
                
        denominator = sum(  np.exp(np.dot(c[k], y_pred_binary[i, k])) for k in range(y_pred_binary[i].shape[0]) if not k in violated_vars_indices  )
        
        weights /= denominator    
        
        instance_weights.append(weights)
    
    
    # y_pred is of shape (N_samples, N_binary_variables)
    # weights is of shape (N_samples, N_binary_variables)
    # each element in weights is the weight for the corresponding element in y_pred
    # multiply the weights by the loss
    
    grad = y_pred - y_true
    hess = y_pred * (1.0 - y_pred)
    
    # multiply the weights by the gradient and hessian
    instance_weights = np.array(instance_weights)
    grad = np.multiply(grad, instance_weights)
    hess = np.multiply(hess, instance_weights)
    
    grad = grad.reshape(-1, 1)
    hess = hess.reshape(-1, 1)

    return grad, hess

# Define a wrapper function that takes only y_true and y_pred as arguments
def custom_obj_wrapper_train(y_true: np.ndarray, y_pred: np.ndarray):
    # reshape y_true to be of shape (N_samples, N_binary_variables)
    y_true = y_true.reshape(y_pred.shape)
    return custom_obj(model_files, indices, train_indices, y_true, y_pred)

# Initialize an XGBClassifier model with the custom objective function
   

In [32]:
xgbmodel = XGBClassifier(objective=custom_obj_wrapper_train, tree_method="hist")

In [33]:
y_train = y_train.astype(np.int)
xgbmodel.fit(X_train, y_train)

/tmp/ipykernel_22975/3213858680.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train = y_train.astype(np.int)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-18

In [76]:
# save the feasibility constrained model
xgbmodel.save_model("Models/Tabular/xgbmodel_feasibility_constrained.json")

/home/jinyi12/miniconda3/envs/optimization/lib/python3.10/site-packages/xgboost/sklearn.py:761: UserWarning: objective is not saved in Scikit-Learn meta.
  warnings.warn(


In [82]:
# load original xgboost model
xgbmodel_ori = XGBClassifier(tree_method="hist")
xgbmodel_ori.load_model("Models/Tabular/xgboost_model_corlat.json")

In [83]:
# predict on test data and calculate average number of infeasible assignments
y_pred = xgbmodel.predict(X_test)
y_pred_ori = xgbmodel_ori.predict(X_test)

In [ ]:
# calculate the average number of infeasible assignments
gurobi_env = gb.Env()
gurobi_env.setParam("OutputFlag", 0)

n_infeasible = 0

# Compute the weights for each training instance
for i in range(y_pred_ori.shape[0]):
    
    model = gb.read("instances/mip/data/COR-LAT/" + model_files[test_indices[i]], env=gurobi_env)
    
    modelVars = model.getVars()
    
    instanceBinaryIndices = indices[test_indices[i]]

    # need to relax the binary variables to continuous variables with bounds of 0 and 1, we can use the setAttr method to change their vtype attribute
    for j in range(len(instanceBinaryIndices)):
        modelVars[instanceBinaryIndices[j]].setAttr("VType", "C")

        # for each index in firstInstanceTestBinaryIndices, set the value of the corresponding variable to the value predicted by xgboost
        modelVars[instanceBinaryIndices[j]].setAttr("LB", y_pred_ori[i, j])
        modelVars[instanceBinaryIndices[j]].setAttr("UB", y_pred_ori[i, j])
    
    
    # Compute the IIS to find the list of violated constraints and variables
    try:
        model.computeIIS()
        infeasible_flag = True
    except:
        infeasible_flag = False
        continue
    
    if infeasible_flag:
        # count the number of violated variables
        for j, v in enumerate(model.getVars()):
            # violated
            if (v.IISLB > 0 or v.IISUB > 0) and j in instanceBinaryIndices:
                n_infeasible += 1
        
    print(n_infeasible)
    
print("Average number of infeasible assignments: ", n_infeasible / y_pred.shape[0])

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, inputs, targets):
        loss = -targets * torch.log(inputs) - (1 - targets) * torch.log(1 - inputs)
        return torch.mean(loss)

In [ ]:
def get_feasibility_constrain_weights(model_files, indices, train_indices, y_true, y_pred):
    
    instance_weights = []
    
    gurobi_env = gb.Env()
    gurobi_env.setParam("OutputFlag", 0)
    
    # convert logit predictions to probabilities
    y_pred = torch.sigmoid(y_pred)

    # convert predictions of N_samples, N_variables to binary
    y_pred_binary = torch.where(y_pred > 0.5, 1, 0)
    
    # Compute the weights for each training instance
    for i in range(y_true.shape[0]):
        
        model = gb.read("instances/mip/data/COR-LAT/" + model_files[i], env=gurobi_env)
        
        modelVars = model.getVars()
        
        instanceBinaryIndices = indices[i]

        # need to relax the binary variables to continuous variables with bounds of 0 and 1, we can use the setAttr method to change their vtype attribute
        for j in range(len(instanceBinaryIndices)):
            modelVars[instanceBinaryIndices[j]].setAttr("VType", "C")

            # for each index in instanceBinaryIndices, set the value of the corresponding variable to the value predicted by the neural network
            modelVars[instanceBinaryIndices[j]].setAttr("LB", y_pred_binary[i, j].item())
            modelVars[instanceBinaryIndices[j]].setAttr("UB", y_pred_binary[i, j].item())
        
        
        # Compute the IIS to find the list of violated constraints and variables
        model.computeIIS()
        
        # Initialize the weights
        weights = np.zeros_like(y_true[i].numpy())
        
        c = model.getAttr("Obj", model.getVars())
        
        # get violated variables indices
        violated_vars_indices = [k for k, v in enumerate(model.getVars()) if (v.IISLB > 0 or v.IISUB > 0) and k in instanceBinaryIndices]
        
        for j, v in enumerate(model.getVars()):
            # not violated
            if (v.IISLB == 0 and v.IISUB == 0) and j in instanceBinaryIndices:
                weights[j] = np.exp(np.dot(c[j], y_pred_binary[i, j].item()))
        
                
        denominator = sum(  np.exp(np.dot(c[k], y_pred_binary[i, k].item())) for k in range(y_pred_binary[i].shape[0]) if not k in violated_vars_indices  )
        
        weights /= denominator    
        
        instance_weights.append(weights)    
        
    instance_weights = np.array(instance_weights)
    
    return np.array(instance_weights)
        
        

In [ ]:
# custom loss for neural network
def custom_loss(model_files: list, indices, y_pred: torch.Tensor, y_true: torch.Tensor):
    
    loss_fn = nn.BCEWithLogitsLoss(reduction='none')
    
    instance_weights = get_feasibility_constrain_weights(model_files, indices, train_indices, y_true, y_pred)
    
    loss = loss_fn(y_pred, y_true)
    
    # multiply the weights by the loss
    loss = np.multiply(loss.numpy(), instance_weights)
    
    return torch.tensor(loss.mean())